In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import csv
import re

musinsa_review = pd.DataFrame(columns = ['Nickname','Product_Name','Body_Size','Information','Review'])

# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

#브라우저 꺼짐 방지 
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
browser = webdriver.Chrome(options=chrome_options)

# 웹페이지 해당 주소 이동
browser.implicitly_wait(5) 
browser.maximize_window()
browser.get("https://www.musinsa.com/categories/item/001005?d_cat_cd=001005&brand=&list_kind=small&sort=emt_high&sub_sort=&page=2&display_cnt=90&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=")
# 로딩이 끝날 때 까지 5초 동안 기다려주는 것이다.

# 스크롤 횟수 지정
scroll_count = 1

# 지정된 스크롤 횟수만큼 반복문을 실행시켜주는 것이다. 
for _ in range(scroll_count):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

time.sleep(2)

items = browser.find_element(By.CSS_SELECTOR,'.list-box')
html_content = items.get_attribute('innerHTML')
browser.quit()

soup = BeautifulSoup(html_content, 'html.parser')
a_tags = soup.find_all('a')

href_list = []

for a in a_tags:
    href = a.get('href', '')
    if 'goods' in href:
        if href.startswith('//'):
            href = 'http:' + href
        href_list.append(href)
        
for href in href_list[40:50]:    

    #브라우저 꺼짐 방지 
    chrome_options = Options()
    chrome_options.add_experimental_option("detach", True)

    # 불필요한 에러 메시지 없애기
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
    browser = webdriver.Chrome(options=chrome_options)

    # 웹페이지 해당 주소 이동
    browser.implicitly_wait(5) 
    browser.maximize_window()
    browser.get(href)
    
    # 2초 정도 기다리기 
    time.sleep(2)
    
    # 해당상품의 html을 가져옴 
    
    # size 정보 담기
    # size_list = browser.find_elements(By.CSS_SELECTOR, '.ifqFgm')
    # more_list = driver.find_elements(By.CSS_SELECTOR, ".css-1aa4nqt")
    
    # 일반 후기 클릭 
    general = browser.find_element(By.CSS_SELECTOR,'#estimate_goods')
    general.click()
    time.sleep(2)
    
    # musinsa_review = pd.DataFrame(columns = ['Nickname','Product_Name','Body_Size','Information','Review'])
    
    page = 20 # 페이지 지정
    
    for p in range(1,page + 1):        
        page_bar = browser.find_elements(By.CSS_SELECTOR,'.paging-btn')
        
        for button in page_bar:
            time.sleep(3)
            if button.text == str(p):
                button.click()
                break 
                
        time.sleep(2)
        
        for review_element in browser.find_elements(By.CSS_SELECTOR, '.review-list'):
            try:
                nickname_element = review_element.find_element(By.CSS_SELECTOR, '.review-profile__name').text
                product_element = review_element.find_element(By.CSS_SELECTOR,'.review-goods-information__name').text
                bodysize_element = review_element.find_element(By.CSS_SELECTOR, '.review-profile__body_information').text
                information_element = review_element.find_element(By.CSS_SELECTOR, '.review-goods-information__option').text
                review_text = review_element.find_element(By.CSS_SELECTOR, '.review-contents__text').text

                print("Nickname:", nickname_element)
                print("Product name",product_element)
                print("Body Size:", bodysize_element)
                print('information_element:',information_element)
                print("Review Text:", review_text)

                musinsa_review = musinsa_review.append({'Nickname': nickname_element,
                                                        'Product_Name':product_element,
                                                        'Body_Size': bodysize_element,
                                                        'Information': information_element,
                                                        'Review': review_text}, ignore_index=True)

            except NoSuchElementException:
                pass
            
        time.sleep(2)
            
        if p % 5 == 0: # 5의 배수일 때 다음5페이지 버튼 클릭 
            browser.find_elements(By.CSS_SELECTOR,'.paging-btn')[7].click()
            time.sleep(2)
            
    browser.quit()

    # 클릭해야 하는 페이지도 다 저장을 해둬야 함
    # 첫 페이지에서 다 추출하고 그 다음 페이지를 클릭하여 리뷰를 넣어준다. 
    # 5페이지의 리뷰를 다 클롤링 했으면, >를 눌러주는 것이다. 
    # 다시 위와 같은 과정을 반복해주는데 총 200번 정도 진행을 해준다. 
    
musinsa_review.to_excel("musinsa_2_50.xlsx", index=False)